In [1]:
import numpy as np
import re
import os

In [242]:
class Ifile():
    
    ALLOWED_KEYS = {'RUN_TYPE', 'CHARGE', 'MULTIPLICITY', 'FUNCTIONAL', 'POTENTIAL_FILE_NAME', 'PSEUDOPOTENTIAL', 'BASIS_SET_FILE_NAME', 'BASIS_SET', 'D3', 'STEPS', 'TIMESTEP', 'CELL_XYZ', 'TEMPERATURE', 'SEED', 'PERIODIC'}
    
    def __init__(self, path, user_input: dict):
        self.path = path
        
        # Default Values
        self.RUN_TYPE = 'MD'
        self.TEMPERATURE = 300
        self.PERIODIC = 'XYZ'
        self.BASIS_SET_FILE_NAME = 'BASIS_SET'
        self.POTENTIAL_FILE_NAME = 'POTENTIAL'
        
        for key, val in user_input.items():
            
            if key in self.ALLOWED_KEYS:
                setattr(self, key, val)
                
            else:
                raise KeyError(f"Unknown parameter: {key}")
        
    def load_geom(self, file):
        
        if re.search('xyz', file):
            
            xyz = np.loadtxt(file, delimiter=None, dtype=str, skiprows=2)
            atoms = xyz[:,0]
            kinds = np.unique(atoms)
            
            self.xyz = xyz
            self.atoms = atoms
            self.kinds = kinds
            self.file = file

            
    def read_keywords(self, keywords, indent=1):
        tab = "\t" * indent
        
        for key, val in keywords.items():
            
            if isinstance(val, list):
                
                # Only run this part of the reader if the list only contains dictionaries. (Otherwise it will loop over the list again.)
                if all(isinstance(item, dict) for item in val):
                    for section in val:
                        print(f"{tab}&{key}")
                        self.read_keywords(section, indent + 1)
                        print(f"{tab}&END {key}")

                continue
            
            if isinstance(val, dict):
                
                print(f"{tab}&{key}")
                self.read_keywords(val, indent=indent+1)
                print(f"{tab}&END {key}")
                
            else:
                print(f"{tab}{key} {val}")
                
    def build_kinds(self):
        '''
        Reads your XYZ file and level of theory to build the kinds section in your CP2K input. 
        '''
        
        kinds_section = []
        
        for at in self.kinds:
            kind = {
                'ELEMENT': at,
                'BASIS_SET': self.BASIS_SET,
                'POTENTIAL': self.PSEUDOPOTENTIAL
            }
            
            kinds_section.append(kind)
            
        return kinds_section
        
    def build_subsys(self):
        '''
        Constructs the SUBSYS section based on user defined inputs. 
        '''
        
        SUBSYS = {
            'CELL': {
                'ABC': self.CELL_XYZ, # USER DEF
                'PERIODIC': self.PERIODIC, # USER DEF
            },

            'TOPOLOGY': {
                'COORD_FILE_NAME': self.file,
                'COORD_FILE_FORMAT': 'XYZ',
                'CENTER_COORDINATES': {
                    'CENTER_POINT': '0 0 0'
                },

            },

            'KIND': self.build_kinds() # AUTOGEN

        }
        
        return SUBSYS
        
    def build_dft(self):
        '''
        Constructs the DFT section based on user defined inputs. 
        
        Right now I'm lazy and D3 empiricial dispersion cannot be turned off.
        
        '''
        
        DFT = {
            'BASIS_SET_FILE_NAME': self.BASIS_SET_FILE_NAME, # USER DEF
            'POTENTIAL_FILE_NAME': self.POTENTIAL_FILE_NAME, # USER DEF
            'CHARGE': self.CHARGE,
            'MULTIPLICITY': self.MULTIPLICITY,

            'MGRID': {
                'CUTOFF': '280', # May need to be converged.
                'NGRIDS': '5'
            },

            'QS': {
                'METHOD': 'GPW',
                'EPS_DEFAULT': '1.0E-10',
                'EXTRAPOLATION': 'ASPC',
            },

            'SCF': {

                'MAX_SCF': '100',
                'EPS_SCF': '1.0E-6',

                'OT': {
                    'PRECONDITIONER': 'FULL_SINGLE_INVERSE',
                    'MINIMIZER': 'DIIS'
                },

                'OUTER_SCF':{
                    'MAX_SCF': '100',
                    'EPS_SCF': '1.0E-5'
                },
                
            },

            'POISSON': {
                'PERIODIC': self.PERIODIC
            },

            'XC': {
                f"XC_FUNCTIONAL {self.FUNCTIONAL}" : {},
                'VDW_POTENTIAL': {
                    'POTENTIAL_TYPE': 'PAIR_POTENTIAL',
                    'PAIR_POTENTIAL': {
                        'PARAMETER_FILE_NAME': './dftd3.dat',
                        'TYPE': 'DFTD3',
                        'REFERENCE_FUNCTIONAL': self.FUNCTIONAL
                    }
                }
            }
        }
        
        return DFT
        
    def build_motion(self):
        
        if self.RUN_TYPE == 'MD':
            
            MOTION = {
                'CELL_OPT': {
                    'TYPE': 'DIRECT_CELL_OPT',
                    'MAX_ITER': '1000'
                },

                'GEO_OPT': {
                    'OPTIMIZER': 'BDGS',
                    'MAX_ITER': '2000',
                    'MAX_DR': '0.003'
                },
                
                'MD': {
                    'ENSEMBLE': 'NVT',
                    'TEMPERATURE': self.TEMPERATURE,
                    'TIMESTEP': self.TIMESTEP,
                    'STEPS': self.STEPS,
                    
                    
                }

            }
            
        return MOTION
                
    def build_section(self, name, keywords):
        '''
        Takes in dictionary to build CP2K input sections. Works with nested dictionaries.
        '''
        
        print(f"&{name}")
        self.read_keywords(keywords)
        print(f"&END {name}")
        
    def construct_input(self):
        
        GLOBAL = {
            'RUN_TYPE': self.RUN_TYPE
        }
        
        FORCE_EVAL = {
            'METHOD': 'Quickstep',
            'STRESS_TENSOR': 'ANALYTICAL',
            'DFT': self.build_dft(),
            'SUBSYS': self.build_subsys()
        }
        
        MOTION = self.build_motion()
        
        self.build_section('GLOBAL', GLOBAL)
        self.build_section('FORCE_EVAL', FORCE_EVAL)
        self.build_section('MOTION', MOTION)
    
            
        

In [244]:
USER_INPUT = {'RUN_TYPE': 'MD',
              'CHARGE': 0,
              'MULTIPLICITY': 1,
              
              'FUNCTIONAL': 'PBE',
              'PSEUDOPOTENTIAL': 'GTH-PBE',
              'POTENTIAL_FILE_NAME': 'GTH_POTENTIALS',
              'BASIS_SET': 'DZVP-MOLOPT-SR-GTH',
              'BASIS_SET_FILE_NAME': 'BASIS_MOLOPT',
              'D3': True,
              
              'STEPS': '10000',
              'TIMESTEP': '0.5',
              'CELL_XYZ': '10 10 10' 
             
             }

cp2k_inp = Ifile(os.getcwd(), USER_INPUT)
cp2k_inp.load_geom('geometry.xyz')

cp2k_inp.construct_input()


&GLOBAL
	RUN_TYPE MD
&END GLOBAL
&FORCE_EVAL
	METHOD Quickstep
	STRESS_TENSOR ANALYTICAL
	&DFT
		BASIS_SET_FILE_NAME BASIS_MOLOPT
		POTENTIAL_FILE_NAME GTH_POTENTIALS
		CHARGE 0
		MULTIPLICITY 1
		&MGRID
			CUTOFF 280
			NGRIDS 5
		&END MGRID
		&QS
			METHOD GPW
			EPS_DEFAULT 1.0E-10
			EXTRAPOLATION ASPC
		&END QS
		&SCF
			MAX_SCF 100
			EPS_SCF 1.0E-6
			&OT
				PRECONDITIONER FULL_SINGLE_INVERSE
				MINIMIZER DIIS
			&END OT
			&OUTER_SCF
				MAX_SCF 100
				EPS_SCF 1.0E-5
			&END OUTER_SCF
		&END SCF
		&POISSON
			PERIODIC XYZ
		&END POISSON
		&XC
			&XC_FUNCTIONAL PBE
			&END XC_FUNCTIONAL PBE
			&VDW_POTENTIAL
				POTENTIAL_TYPE PAIR_POTENTIAL
				&PAIR_POTENTIAL
					PARAMETER_FILE_NAME ./dftd3.dat
					TYPE DFTD3
					REFERENCE_FUNCTIONAL PBE
				&END PAIR_POTENTIAL
			&END VDW_POTENTIAL
		&END XC
	&END DFT
	&SUBSYS
		&CELL
			ABC 10 10 10
			PERIODIC XYZ
		&END CELL
		&TOPOLOGY
			COORD_FILE_NAME geometry.xyz
			COORD_FILE_FORMAT XYZ
			&CENTER_COORDINATES
				CENTER_POINT 0